In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import nltk
nltk.download('punkt')
import spacy

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to C:\Users\Anas
[nltk_data]     Laptop\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
train = pd.read_csv('train1.csv')
test = pd.read_csv('test1.csv')

In [3]:
df = train.head(100000)

In [4]:
df.head()

,date,flag,user,text,id,target
0,Wed Jun 03 03:47:24 PDT 2009,NO_QUERY,mayhemstudios,"@JennaMadison awww, thanks . I like you too, a...",1325379,1
1,Mon Jun 01 05:54:18 PDT 2009,NO_QUERY,ruralmama,&quot;The man&quot; is gone for a week. Pouting!,275675,0
2,Wed Jun 17 17:34:38 PDT 2009,NO_QUERY,princesslaylah,stupid itunes keeps crashing so i can't update...,583702,0
3,Fri May 29 11:43:58 PDT 2009,NO_QUERY,candace_lea,is ready for the weekend!,1058019,1
4,Mon Jun 15 12:45:11 PDT 2009,NO_QUERY,joluvs2shop,@schofe must be great getting all the freebies...,1545430,1


In [5]:
df.shape

(100000, 6)

In [6]:
type(df)

pandas.core.frame.DataFrame

In [7]:
df.drop(['date','flag','user'],axis=1 ,inplace=True)

C:\Users\Anas Laptop\AppData\Local\Temp\ipykernel_23788\3527098657.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['date','flag','user'],axis=1 ,inplace=True)


In [8]:
df.head()

,text,id,target
0,"@JennaMadison awww, thanks . I like you too, a...",1325379,1
1,&quot;The man&quot; is gone for a week. Pouting!,275675,0
2,stupid itunes keeps crashing so i can't update...,583702,0
3,is ready for the weekend!,1058019,1
4,@schofe must be great getting all the freebies...,1545430,1


In [9]:
url_pattern = re.compile(r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)")
df['text'] = df['text'].apply(lambda x: re.sub(url_pattern, 'URL', x))

user_pattern = re.compile(r'@[\w]+')
df['text'] = df['text'].apply(lambda x: re.sub(user_pattern, 'USER', x))

# The Thrid step convert any Emoticon keyboard shortcuts to Words
emojis = {'(:': 'Smiley face', '(-:': 'Smiley face', '[:': 'Content face',
         '(;': 'Winky face', '(-;': 'Winky face', 'D:': 'Thrilled face', 'D-:': 'Thrilled face',
         'P;': 'Goofy face', 'P-;': 'Goofy face', 'P:': 'Silly face', 'P-:': 'Silly face',
         '(8': 'Cool guy face', '(-8': 'Cool guy face', '|:':'Blank face', '|-:': 'Blank face',
         '(:': 'Sad face', '(-:':'Sad face', 'O_o':'Grossed out face', 'O.o':'Grossed out face',
         '\:': 'Sick face', '\-:': 'Sick face', 'O:': 'Surprised face', 'O-:': 'Surprised face',
         'O_O': 'Tweak face', 'O.O': 'Tweak face', 'o_o': 'Grossed out face', '(#':'Poundie',
         '(-#': 'Poundie', '^.^': 'Nerdie Poundie', '^_^':'Nerdie Poundie', '^^':'Nerdie Poundie',
         'x:': 'Kissy face', 'x-:': 'Kissy face','<3': 'Heart', ')#': 'Frowndie', ')-#': 'Frowndie',
         '(m': 'Facepalm', '(-m': 'Facepalm', 'Dx': 'Crying laughing', 'DX': 'Crying laughing',
         ")':": 'Crying face', ")-':": 'Crying face',':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad',
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed',
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink',
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}

for emoji in emojis.keys():
    df['text'].replace(emoji,'EMOJI'+emojis[emoji])
    
# The Fourth step removing Non-Alphabets
alpha_pattern = "[^a-zA-Z0-9\s]"
df['text'] = df['text'].apply(lambda x: re.sub(alpha_pattern, "", x))

# The Fivth step removing Consecutive letters than more 3

sequence_pattern   = r"(.)\1\1+"
seqReplace_pattern = r"\1\1"
df['text'] = df['text'].apply(lambda x: re.sub(sequence_pattern, seqReplace_pattern, x))


# The Sixth step convert the text to lower case

df['text'] = df['text'].apply(lambda x: x.lower())

C:\Users\Anas Laptop\AppData\Local\Temp\ipykernel_23788\3384779416.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(lambda x: re.sub(url_pattern, 'URL', x))
C:\Users\Anas Laptop\AppData\Local\Temp\ipykernel_23788\3384779416.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(lambda x: re.sub(user_pattern, 'USER', x))
C:\Users\Anas Laptop\AppData\Local\Temp\ipykernel_23788\3384779416.py:31: SettingWithCopyWarning: 
A value is trying to be set on 

In [10]:
stop_words = set(stopwords.words('english'))

# Function to clean and tokenize text
def clean_tokenize(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove usernames
    text = re.sub(r'@\w+', '', text)
    # Remove special characters and numbers
    text = re.sub(r'[^A-Za-z ]+', '', text)
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stop words+
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

df['cleaned_text'] = df['text'].apply(clean_tokenize)

C:\Users\Anas Laptop\AppData\Local\Temp\ipykernel_23788\289386078.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_text'] = df['text'].apply(clean_tokenize)


In [11]:
df['cleaned_text']

0                        user aww thanks like always funny
1                        quotthe manquot gone week pouting
2        stupid itunes keeps crashing cant update iphon...
3                                            ready weekend
4        user must great getting freebies dont want im ...
                               ...                        
99995             boy supposed love love friend soo angryy
99996    user paid hour fair yes even mopped kitchen fl...
99997              user whoo scrubs borrow six season body
99998       great day rampr glad sun makes things brighter
99999                            mowing lawn weather sucks
Name: cleaned_text, Length: 100000, dtype: object

In [12]:
df.head()

,text,id,target,cleaned_text
0,user aww thanks i like you too always funny,1325379,1,user aww thanks like always funny
1,quotthe manquot is gone for a week pouting,275675,0,quotthe manquot gone week pouting
2,stupid itunes keeps crashing so i cant update ...,583702,0,stupid itunes keeps crashing cant update iphon...
3,is ready for the weekend,1058019,1,ready weekend
4,user must be great getting all the freebies an...,1545430,1,user must great getting freebies dont want im ...


In [13]:
import pandas as pd
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
# Initialize PorterStemmer
ps = PorterStemmer()

# Define a function to stem each word in a text
def stem_text(text):
    # Tokenize the text into words
    words = word_tokenize(text)
    # Stem each word
    stemmed_words = [ps.stem(word) for word in words]
    # Join the stemmed words back into a single string
    return ' '.join(stemmed_words)

# Assuming 'df' is your DataFrame and 'text' is the column with text data
# Apply the function to each row in the 'text' column
df['cleaned_text'] = df['text'].apply(stem_text)


C:\Users\Anas Laptop\AppData\Local\Temp\ipykernel_23788\287628158.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_text'] = df['text'].apply(stem_text)


In [14]:
df.head()

,text,id,target,cleaned_text
0,user aww thanks i like you too always funny,1325379,1,user aww thank i like you too alway funni
1,quotthe manquot is gone for a week pouting,275675,0,quotth manquot is gone for a week pout
2,stupid itunes keeps crashing so i cant update ...,583702,0,stupid itun keep crash so i cant updat my ipho...
3,is ready for the weekend,1058019,1,is readi for the weekend
4,user must be great getting all the freebies an...,1545430,1,user must be great get all the freebi ani you ...


In [15]:
df['cleaned_text'][0:5]

0            user aww thank i like you too alway funni
1               quotth manquot is gone for a week pout
2    stupid itun keep crash so i cant updat my ipho...
3                             is readi for the weekend
4    user must be great get all the freebi ani you ...
Name: cleaned_text, dtype: object

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence


#max_features = 2000
vectorizer = CountVectorizer(max_features=None)
text = csr_matrix(vectorizer.fit_transform(df['cleaned_text']))

In [ ]:
model = Word2Vec(tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)

In [17]:
# #vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(df['text'])
# y = df['target']

# # Splitting the data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# # Model
# model = LogisticRegression()
# model.fit(X_train, y_train)

# # Predictions
# predictions = model.predict(X_test)

# # Evaluation
# print(classification_report(y_test, predictions))

# #####################
# # vectorizer = TfidfVectorizer(max_features=None)
# # X = vectorizer.fit_transform(df['text'])
# # y = df['target']

# # # Split the data
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# # # XGBoost model
# # xgb_model = XGBClassifier()
# # xgb_model.fit(X_train, y_train)

# # # Predictions
# # predictions = xgb_model.predict(X_test)

# # # Evaluation
# # print(classification_report(y_test, predictions))

In [18]:
# X = vectorizer.fit_transform(df['cleaned_text'])
# y = df['target']

# # Splitting the data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# # Model
# model = LogisticRegression()
# model.fit(X_train, y_train)

# # Predictions
# predictions = model.predict(X_test)

# # Evaluation
# print(classification_report(y_test, predictions))

In [19]:
# #vectorizer = TfidfVectorizer()
# vectorizer = TfidfVectorizer(max_features=None)

# X = vectorizer.fit_transform(df['text'])
# y = df['target']

# # Splitting the data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# # Model
# model = LogisticRegression()
# model.fit(X_train, y_train)

# # Predictions
# predictions = model.predict(X_test)

# # Evaluation
# print(classification_report(y_test, predictions))

Logistic Reg  
1- Count
2-tdid

In [20]:
# vectorizer = TfidfVectorizer(max_features=None)
# X = vectorizer.fit_transform(df['text'])
# y = df['target']

# # Split the data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# # XGBoost model
# xgb_model = XGBClassifier()
# xgb_model.fit(X_train, y_train)

# # Predictions
# predictions = xgb_model.predict(X_test)

# # Evaluation
# print(classification_report(y_test, predictions))

In [21]:
# vectorizer = CountVectorizer(max_features=None)
# X = vectorizer.fit_transform(df['text'])
# y = df['target']

# # Split the data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# # XGBoost model
# xgb_model = XGBClassifier()
# xgb_model.fit(X_train, y_train)

# # Predictions
# predictions = xgb_model.predict(X_test)

# # Evaluation
# print(classification_report(y_test, predictions))

In [22]:
# vectorizer = CountVectorizer(max_features=None)
# X = vectorizer.fit_transform(df['text'])
# y = df['target']

# # Split the data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# # XGBoost model
# xgb_model = xgb.XGBClassifier(n_estimators = 5000,  random_state = 42 , max_depth  = 10 ,learning_rate = 0.2
#                            ,colsample_bytree =0.8,subsample = 1 , min_child_weight =10)
# xgb_model.fit(X_train, y_train)

# # Predictions
# predictions = xgb_model.predict(X_test)

# # Evaluation
# print(classification_report(y_test, predictions))

In [23]:
# vectorizer = CountVectorizer(max_features=None)
# X = vectorizer.fit_transform(df['cleaned_text'])
# y = df['target']

# # Split the data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# # XGBoost model
# xgb_model = xgb.XGBClassifier(n_estimators = 5000,  random_state = 42 , max_depth  = 10 ,learning_rate = 0.2
#                            ,colsample_bytree =0.8,subsample = 1 , min_child_weight =10)
# xgb_model.fit(X_train, y_train)

# # Predictions
# predictions = xgb_model.predict(X_test)

# # Evaluation
# print(classification_report(y_test, predictions))

In [24]:

# # Assuming you have new data in a DataFrame called new_data
# new_text_data = test['text']

# new_X = vectorizer.transform(new_text_data)


# new_predictions = xgb_model.predict(new_X)

# # Add a new column 'predicted_target' to test
# #test['predicted_target'] = new_predictions

# # Add a new column 'predicted_target' to new_data
# test['target'] = new_predictions

# # Save the DataFrame to a CSV file
# #test.to_csv('predicted_results.csv', index=False)

In [25]:
sample = pd.read_csv('sample.csv')

Xgbost 
1- tFid 
2- Counter
3-Counter with HP

In [26]:
#sample['target'] = new_predictions

In [27]:
#sample['target']

In [28]:
#sample.to_csv('sample010.csv', index=False)

In [29]:
vectorizer = CountVectorizer(max_features=None)
text = csr_matrix(vectorizer.fit_transform(df['cleaned_text']))
X = vectorizer.fit_transform(df['cleaned_text'])
y = df['target']

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predictions
predictions = model.predict(X_test)

# Evaluation
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.77      0.76      0.77     14827
           1       0.77      0.78      0.78     15173

    accuracy                           0.77     30000
   macro avg       0.77      0.77      0.77     30000
weighted avg       0.77      0.77      0.77     30000



C:\Users\Anas Laptop\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [42]:
import gensim
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

# Tokenize your text (assuming df['cleaned_text'] is already a list of words for each document)
tokenized_sentences = [text.split() for text in df['cleaned_text']]

# Train Word2Vec
vectorizer = gensim.models.Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)

# Function to create averaged word vector for a document
def document_vector(doc):
    # Remove out-of-vocabulary words
    doc = [word for word in doc if word in vectorizer.wv.index_to_key]
    return np.mean(vectorizer.wv[doc], axis=0) if len(doc) > 0 else np.zeros(vectorizer.vector_size)

# Create feature vectors for each document
X = np.array([document_vector(doc) for doc in tokenized_sentences])

y = df['target']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predictions
predictions = model.predict(X_test)

# Evaluation
print(classification_report(y_test, predictions))


              precision    recall  f1-score   support

           0       0.73      0.74      0.73     15017
           1       0.73      0.73      0.73     14983

    accuracy                           0.73     30000
   macro avg       0.73      0.73      0.73     30000
weighted avg       0.73      0.73      0.73     30000



C:\Users\Anas Laptop\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [30]:

# Assuming you have new data in a DataFrame called new_data
new_text_data = test['text']

new_X = vectorizer.transform(new_text_data)


new_predictions = model.predict(new_X)

# Add a new column 'predicted_target' to test
#test['predicted_target'] = new_predictions

# Add a new column 'predicted_target' to new_data
test['target'] = new_predictions

# Save the DataFrame to a CSV file
#test.to_csv('predicted_results.csv', index=False)

In [31]:
sample['target'] = new_predictions

In [32]:
sample['target']

0         1
1         1
2         1
3         1
4         0
         ..
255995    0
255996    1
255997    0
255998    0
255999    0
Name: target, Length: 256000, dtype: int64

In [33]:
sample.to_csv('sample020.csv', index=False)

In [34]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk

# Sample sentences
sentences = ["I enjoy reading books",
             "I love writing code",
             "I read a lot",
             "Programming is fun",
             "Books are great for learning"]

# Tokenizing the sentences
nltk.download('punkt')
tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in sentences]

# Training the Word2Vec model
model = Word2Vec(tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)

# Getting the word vector for a specific word
word_vector = model.wv['books']

# Finding similar words
similar_words = model.wv.most_similar('books', topn=5)

print("Vector representation of 'books':", word_vector)
print("\nWords similar to 'books':", similar_words)

Vector representation of 'books': [-8.6196596e-03  3.6659390e-03  5.1903180e-03  5.7418076e-03
  7.4669695e-03 -6.1675780e-03  1.1056825e-03  6.0472889e-03
 -2.8401462e-03 -6.1733695e-03 -4.1022216e-04 -8.3692092e-03
 -5.5998950e-03  7.1045891e-03  3.3525999e-03  7.2255637e-03
  6.7998939e-03  7.5310115e-03 -3.7893557e-03 -5.6184293e-04
  2.3484002e-03 -4.5189029e-03  8.3890324e-03 -9.8582022e-03
  6.7643207e-03  2.9147104e-03 -4.9326117e-03  4.3977732e-03
 -1.7394728e-03  6.7113303e-03  9.9646291e-03 -4.3627555e-03
 -5.9921597e-04 -5.6956350e-03  3.8510798e-03  2.7867644e-03
  6.8911952e-03  6.1009317e-03  9.5382128e-03  9.2731910e-03
  7.8979973e-03 -6.9897701e-03 -9.1560259e-03 -3.5538655e-04
 -3.0997922e-03  7.8942655e-03  5.9384434e-03 -1.5459724e-03
  1.5108237e-03  1.7897255e-03  7.8173140e-03 -9.5105413e-03
 -2.0546034e-04  3.4690993e-03 -9.3898363e-04  8.3820624e-03
  9.0108793e-03  6.5364111e-03 -7.1155117e-04  7.7104783e-03
 -8.5343765e-03  3.2068116e-03 -4.6381308e-03 -5.08

[nltk_data] Downloading package punkt to C:\Users\Anas
[nltk_data]     Laptop\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
